# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 01 - Parte 2 - Ranking e Modelo Vetorial
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Importações necessárias para a análise dos dados

In [50]:
import pandas as pd
import nltk
import numpy as np
import ast
import re
from unicodedata import normalize
from collections import Counter
import math

### Fazendo download de biblioteca necessária para realizar a tokenização das palavras

In [62]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JohannyLS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True



## Definindo o arquivo com os dados a ser analisado e o gabarito

In [2]:
dataset = pd.read_csv('../data/estadao_noticias_eleicao.csv', sep=',')

#Substituindo linhas com valores NaN por string vazia, para não atrapalhar nas operações
dataset = dataset.replace(np.nan, '', regex=True)

## Tratamento do Gabarito
 Usando a biblioteca **Abstract Syntax Trees** (ast) para converter as strings do gabarito para o objeto lista, junto com a função apply() da biblioteca **Pandas** para aplicar a conversão em todas as linhas

In [3]:
gabarito = pd.read_csv('../data/gabarito.csv')

gabarito['google']        = gabarito['google'].apply(ast.literal_eval)
gabarito['busca_binaria'] = gabarito['busca_binaria'].apply(ast.literal_eval)
gabarito['tf']            = gabarito['tf'].apply(ast.literal_eval)
gabarito['tfidf']         = gabarito['tfidf'].apply(ast.literal_eval)
gabarito['bm25']          = gabarito['bm25'].apply(ast.literal_eval)

## Funções para avaliação das saídas utilizando a métrica Mean Average Precision (MAP)

In [4]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

### Função para remover acentuação das palavras dos documentos
Módulo **re** e **unicodedata** necessários para tratamento de expressões regulares e normalização de string

In [5]:
def remove_acentuacao(palavra):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    texto = normalize('NFKD', palavra).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', palavra)

## Realizando junção dos títulos, subtítulos e conteúdos
Na junção, está sendo removidos todos os acentos para facilitar a criação dos tokens dos dados

In [6]:
documentos = dataset.titulo + " " + dataset.subTitulo + " " + dataset.conteudo
documentos = documentos.apply(lambda palavra: remove_acentuacao(palavra).lower())

## Criação dos docIDs

In [7]:
docIDs = dataset.idNoticia

## Criando os tokens dos dados e a frequência de termos de cada documento da lista de postings
Utilizando a função **Count** da biblioteca **Collections** para realizar a contagem dos termos (frequência)

In [11]:
tokens = documentos.apply(nltk.word_tokenize)
termo_frequency = tokens.apply(Counter)

## Função geradora do índice invertido

In [19]:
indiceInvertido = {}

def gerarIndiceInvertido():
    for i in range(len(tokens)):
        idNoticia = docIDs[i]
        palavras = tokens[i]
    
        for palavra in palavras:
            if palavra not in indiceInvertido:
                indiceInvertido[palavra] = {}
        
            if not indiceInvertido[palavra].get(idNoticia):
                docs = indiceInvertido[palavra]
                docs[idNoticia] = tf[i][palavra]

In [20]:
gerarIndiceInvertido()

## Função para retornar as palavras mais proximas

In [58]:
def busca_aproximada(termo, qtd):
    termos = indiceInvertido.keys()
    return sorted(termos, key=lambda palavra: nltk.edit_distance(termo,palavra))[0:qtd]

## Função para gerar um dicionário com os pesos dos índices

In [21]:
def gerar_dict_pesos(frase, gerador_peso):
    termos = frase.split(" ")
    docs_peso = {}
    
    for i in range(len(termos)):
        termo = termos[i]
        docs = indiceInvertido[termo]
        
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in docs_peso:
                docs_peso[doc_id] = np.array([0 if j != i else gerador_peso(tf) for j in range(len(termos))])
            else:
                doc_vector = docs_peso[doc_id]
                docs_peso[doc_id] = np.array([doc_vector[j] if j != i else gerador_peso(tf) for j in range(len(termos))])
    
    return docs_peso

## Função para gerar um dicionário com vetores binários

In [28]:
def gerar_binario(frase):
    def gerador_peso(tf):
        return 1
    return gerar_dict_pesos(frase, gerador_peso)

## Função para gerar o TF da consulta

In [29]:
def gerar_vetor_tf(frase):
    def gerador_peso(tf):
        return tf
    return gerar_dict_pesos(frase, gerador_peso)

## Função para gerar IDF dos termos

In [52]:
def gerar_vetor_tfidf(frase):
    termos = frase.split(" ")
    vetor_tfidf = np.array([math.log((len(documentos)+1)/len(indiceInvertido[termo])) for termo in termos])
    return vetor_tfidf

## Função para gerar o vetor binário de consulta

In [31]:
def gerar_consulta(frase):
    termos = frase.split(" ")
    vetor = np.array([1 if indiceInvertido.get(termo) else 0 for termo in termos])
    return vetor

## Função para gerar um dicionário com vetores BM25

In [40]:
def gerar_vetor_bm25(frase):
    docs_tf = gerar_vetor_tf(frase)
    k = 5
    vetor_bm25 = {doc_id: np.array([((k+1)*tf)/(tf+k) for tf in vetor_tf]) for doc_id, vetor_tf in docs_tf.items()}
    
    return vetor_bm25

## Função de busca binária

In [45]:
def busca_binaria(frase):
    docs_tf = gerar_binario(frase)
    consulta = gerar_consulta(frase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], consulta), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]

## Função de busca por TF

In [38]:
def busca_tf(frase):
    docs_tf = gerar_vetor_tf(frase)
    query = gerar_consulta(frase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]

## Função de busca por TF IDF

In [37]:
def busca_tf_idf(frase):
    doc_tf = gerar_vetor_tf(frase)
    doc_idf = gerar_vetor_tfidf(frase)
    doc_rank = sorted(list(doc_tf.items()), key=lambda doc: np.dot(doc[1], doc_idf), reverse=True)[:5]
    return [doc[0] for doc in doc_rank]

## Função de busca por BM25

In [41]:
def busca_bm25(frase):
    doc_bm25 = gerar_vetor_bm25(frase)
    query = gerar_consulta(frase)
    doc_rank = sorted(list(doc_bm25.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)[:5]
    return [doc[0] for doc in doc_rank]

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Realizando avaliações através da Mean Average Precision

## Avaliação da precisão da busca binária

In [48]:
busca_bin  = [busca_binaria(remove_acentuacao(frase)) for frase in gabarito.str_busca]

print("Local  = %.4f" %(mapk(gabarito.busca_binaria, busca_bin, k=5)))
print("Google = %.4f" %(mapk(gabarito.google, busca_bin, k=5)))

Local  = 0.2400
Google = 0.0400


## Avaliação da precisão da busca por TF

In [54]:
busca_term_freq = [busca_tf(remove_acentuacao(frase)) for frase in gabarito.str_busca]

print("Local  = %.4f" %(mapk(gabarito.tf, busca_term_freq, k=5)))
print("Google = %.4f" %(mapk(gabarito.google, busca_term_freq, k=5)))

Local  = 0.6040
Google = 0.0480


## Avaliação da precisão da busca por TF IDF

In [56]:
busca_tfidf  = [busca_tf_idf(remove_acentuacao(frase)) for frase in gabarito.str_busca]
print("Local  = %.4f" %(mapk(gabarito.tfidf, busca_tfidf, k=5)))
print("Google = %.4f" %(mapk(gabarito.google, busca_tfidf, k=5)))

Local  = 0.5920
Google = 0.0600


# Consultas

### Busca: 'segundo turno'

In [67]:
print("Busca Binária   :", busca_binaria('segundo turno'))
print("Busca por TF    :", busca_tf('segundo turno'))
print("Busca por TF IDF:", busca_tf_idf('segundo turno'))
print("Busca por BM25  :", busca_bm25('segundo turno'))

Busca Binária   : [1, 7, 13, 26, 69]
Busca por TF    : [2744, 7, 2112, 7672, 2388]
Busca por TF IDF: [2744, 2112, 7672, 1235, 2388]
Busca por BM25  : [2744, 2112, 7672, 2388, 2178]


### Busca: 'lava jato'

In [68]:
print("Busca Binária   :", busca_binaria('lava jato'))
print("Busca por TF    :", busca_tf('lava jato'))
print("Busca por TF IDF:", busca_tf_idf('lava jato'))
print("Busca por BM25  :", busca_bm25('lava jato'))

Busca Binária   : [3, 13, 15, 27, 43]
Busca por TF    : [163, 353, 6942, 2807, 127]
Busca por TF IDF: [163, 353, 6942, 2807, 127]
Busca por BM25  : [163, 353, 6942, 2807, 127]


### Busca: 'projeto de lei'

In [70]:
print("Busca Binária   :", busca_binaria('projeto de lei'))
print("Busca por TF    :", busca_tf('projeto de lei'))
print("Busca por TF IDF:", busca_tf_idf('projeto de lei'))
print("Busca por BM25  :", busca_bm25('projeto de lei'))

Busca Binária   : [7, 10, 25, 38, 56]
Busca por TF    : [7, 155, 6554, 3942, 7017]
Busca por TF IDF: [7017, 5129, 6096, 2853, 155]
Busca por BM25  : [2853, 7017, 2232, 3171, 6461]


### Busca: 'compra de voto'

In [72]:
print("Busca Binária   :", busca_binaria('compra de voto'))
print("Busca por TF    :", busca_tf('compra de voto'))
print("Busca por TF IDF:", busca_tf_idf('compra de voto'))
print("Busca por BM25  :", busca_bm25('compra de voto'))

Busca Binária   : [82, 347, 553, 748, 854]
Busca por TF    : [7, 155, 6554, 3942, 7017]
Busca por TF IDF: [173, 2047, 7017, 7343, 7]
Busca por BM25  : [2047, 2178, 7017, 2200, 7343]


### Busca: 'ministério público'

In [88]:
print("Busca Binária   :", busca_binaria('publico'))
print("Busca por TF    :", busca_tf('compra de voto'))
print("Busca por TF IDF:", busca_tf_idf('compra de voto'))
print("Busca por BM25  :", busca_bm25('compra de voto'))

Busca Binária   : [578, 845, 847, 874, 1069]
Busca por TF    : [7, 155, 6554, 3942, 7017]
Busca por TF IDF: [173, 2047, 7017, 7343, 7]
Busca por BM25  : [2047, 2178, 7017, 2200, 7343]


In [89]:
print(busca_aproximada('ministerio publico',10))

['ministerial', 'ministeriais', 'ministeri', 'interrompido', 'ministrou', 'ministro', 'interioriza', 'ministros', 'interrompendo', 'interrompida']
